# Import and Install Dependencies

In [4]:
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt
import time
import mediapipe as mp
%matplotlib inline

# Key points using Media pipe holistic

In [5]:
mp_holistic = mp.solutions.holistic #holistic model
mp_drawing = mp.solutions.drawing_utils #drawing utilities

In [6]:
def mediapipe_detection(image,model):
    image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB) #color conversion bgr to rgb
    image.flags.writeable=False  #image is no longer writeable
    results=model.process(image)  #make prediction
    image.flags.writeable=True  #image is now writable
    image=cv2.cvtColor(image,cv2.COLOR_RGB2BGR) #color conversion rgb to bgr
    return image,results

In [7]:
def draw_landmarks(image,results):
    mp_drawing.draw_landmarks(image,results.face_landmarks,mp_holistic.FACEMESH_CONTOURS)
    mp_drawing.draw_landmarks(image,results.pose_landmarks,mp_holistic.POSE_CONNECTIONS)
    mp_drawing.draw_landmarks(image,results.left_hand_landmarks,mp_holistic.HAND_CONNECTIONS)
    mp_drawing.draw_landmarks(image,results.right_hand_landmarks,mp_holistic.HAND_CONNECTIONS)

In [30]:
mp_holistic.POSE_CONNECTIONS

frozenset({(0, 1),
           (0, 4),
           (1, 2),
           (2, 3),
           (3, 7),
           (4, 5),
           (5, 6),
           (6, 8),
           (9, 10),
           (11, 12),
           (11, 13),
           (11, 23),
           (12, 14),
           (12, 24),
           (13, 15),
           (14, 16),
           (15, 17),
           (15, 19),
           (15, 21),
           (16, 18),
           (16, 20),
           (16, 22),
           (17, 19),
           (18, 20),
           (23, 24),
           (23, 25),
           (24, 26),
           (25, 27),
           (26, 28),
           (27, 29),
           (27, 31),
           (28, 30),
           (28, 32),
           (29, 31),
           (30, 32)})

In [42]:
mp_drawing.draw_landmarks??

In [8]:

def draw_styled_landmarks(image, results):
    # Draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS, 
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             ) 
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

In [39]:
cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5,min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        ret, frame = cap.read()

        #make detections
        image,results = mediapipe_detection(frame,holistic)
        
        #draw landmarks
        draw_landmarks(image,results)
        draw_styled_landmarks(image,results)
        #show to screen
        cv2.imshow('OpenCV Feed',image)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
cap.release()
cv2.destroyAllWindows()

# Extract key point values

In [61]:
pose=[]
for res in results.pose_landmarks.landmark:
    test = np.array([res.x,res.y,res.z,res.visibility])
    pose.append(test)
pose=np.array(pose)
pose

array([[ 0.62738407,  0.65827847, -0.9792062 ,  0.98509121],
       [ 0.62240237,  0.60336536, -1.02160907,  0.98840696],
       [ 0.62652594,  0.60119128, -1.02091527,  0.99005473],
       [ 0.63028419,  0.59907144, -1.02030289,  0.99067456],
       [ 0.59998637,  0.60407436, -1.07833529,  0.9834584 ],
       [ 0.58819985,  0.60198563, -1.0781424 ,  0.98056513],
       [ 0.57521093,  0.59986883, -1.07753515,  0.97744304],
       [ 0.61308789,  0.58903968, -0.87197101,  0.99301785],
       [ 0.52874792,  0.59178257, -1.1000551 ,  0.9827826 ],
       [ 0.6371733 ,  0.68621284, -0.87402999,  0.98673302],
       [ 0.61450815,  0.68557733, -0.95854539,  0.98036945],
       [ 0.61774516,  0.74592894, -0.57945627,  0.95677769],
       [ 0.40440011,  0.73464864, -0.92535782,  0.93491191],
       [ 0.73955256,  0.84161478, -0.3076511 ,  0.50960165],
       [ 0.49151525,  0.72100747, -0.76492661,  0.50936347],
       [ 0.76415503,  0.85695589, -0.21669266,  0.22521958],
       [ 0.64134109,  0.

In [62]:
pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(132)
face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(1404)
lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)

In [63]:
face

array([0., 0., 0., ..., 0., 0., 0.])

In [66]:
pose

array([ 0.62738407,  0.65827847, -0.9792062 ,  0.98509121,  0.62240237,
        0.60336536, -1.02160907,  0.98840696,  0.62652594,  0.60119128,
       -1.02091527,  0.99005473,  0.63028419,  0.59907144, -1.02030289,
        0.99067456,  0.59998637,  0.60407436, -1.07833529,  0.9834584 ,
        0.58819985,  0.60198563, -1.0781424 ,  0.98056513,  0.57521093,
        0.59986883, -1.07753515,  0.97744304,  0.61308789,  0.58903968,
       -0.87197101,  0.99301785,  0.52874792,  0.59178257, -1.1000551 ,
        0.9827826 ,  0.6371733 ,  0.68621284, -0.87402999,  0.98673302,
        0.61450815,  0.68557733, -0.95854539,  0.98036945,  0.61774516,
        0.74592894, -0.57945627,  0.95677769,  0.40440011,  0.73464864,
       -0.92535782,  0.93491191,  0.73955256,  0.84161478, -0.3076511 ,
        0.50960165,  0.49151525,  0.72100747, -0.76492661,  0.50936347,
        0.76415503,  0.85695589, -0.21669266,  0.22521958,  0.64134109,
        0.65858418, -0.39007279,  0.25301343,  0.77708471,  0.86

In [9]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

In [68]:
extract_keypoints(results).shape

(1662,)

In [69]:
468*3+33*4+21*3+21*3

1662

# setup folders for data collection

In [13]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('MP_Data') 

# Actions that we try to detect
actions = np.array(['hello', 'thanks', 'iloveyou'])

# Thirty videos worth of data
no_sequences = 30

# Videos are going to be 30 frames in length
sequence_length = 30

# Folder start
start_folder = 30


In [73]:
#hello 
##0
##1
#...
##29

#thanks

#I love you

In [12]:
for action in actions: 
#     dirmax = np.max(np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int))
    for sequence in range(no_sequences):
        try: 
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

NameError: name 'actions' is not defined

# Collect keypoint values for training and testing

In [79]:
cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5,min_tracking_confidence=0.5) as holistic:
    for action in actions:
        for sequence in range(no_sequences):
            for frame_num in range(sequence_length):
                
                ret, frame = cap.read()

                #make detections
                image,results = mediapipe_detection(frame,holistic)

                #draw landmarks
                draw_landmarks(image,results)
                draw_styled_landmarks(image,results)
                
                if frame_num == 0:
                    cv2.putText(image,'STARTING COLLECTION',(120,200),
                               cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),4,cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                    cv2.waitKey(500)
                else: 
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                
                # NEW Export keypoints
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)
                #show to screen
                cv2.imshow('OpenCV Feed',image)
                
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
cap.release()
cv2.destroyAllWindows()

# Preprocess Data and create labels and features

In [10]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [14]:
label_map = {label:num for num, label in enumerate(actions)}

In [15]:
label_map

{'hello': 0, 'thanks': 1, 'iloveyou': 2}

In [11]:
sequences, labels = [], []
for action in actions:
    for sequence in np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [12]:
np.array(sequences).shape

(90, 30, 1662)

In [13]:
np.array(labels).shape

(90,)

In [14]:
X = np.array(sequences)

In [15]:
y = to_categorical(labels).astype(int)
y

array([[1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0,

In [16]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.05)

# Build and train LSTM Neural network

In [17]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense
from tensorflow.keras.callbacks import TensorBoard

In [18]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [14]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [15]:
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['categorical_accuracy'])

In [16]:
model.fit(X_train, y_train, epochs=2000, callbacks=[tb_callback])

Epoch 1/2000
3/3 [==============================] - 6s 530ms/step - loss: 4.1600 - categorical_accuracy: 0.3529
Epoch 2/2000
3/3 [==============================] - 2s 539ms/step - loss: 1.2193 - categorical_accuracy: 0.2706
Epoch 3/2000
3/3 [==============================] - 2s 512ms/step - loss: 1.1752 - categorical_accuracy: 0.2706
Epoch 4/2000
3/3 [==============================] - 2s 531ms/step - loss: 1.0856 - categorical_accuracy: 0.4353
Epoch 5/2000
3/3 [==============================] - 2s 525ms/step - loss: 1.2912 - categorical_accuracy: 0.2824
Epoch 6/2000
3/3 [==============================] - 2s 544ms/step - loss: 1.3025 - categorical_accuracy: 0.4000
Epoch 7/2000
3/3 [==============================] - 2s 530ms/step - loss: 1.1050 - categorical_accuracy: 0.4000
Epoch 8/2000
3/3 [==============================] - 2s 526ms/step - loss: 0.9737 - categorical_accuracy: 0.4941
Epoch 9/2000
3/3 [==============================] - 2s 521ms/step - loss: 0.8417 - categorical_accuracy:

3/3 [==============================] - 2s 651ms/step - loss: 0.4413 - categorical_accuracy: 0.6824
Epoch 146/2000
3/3 [==============================] - 2s 604ms/step - loss: 0.4182 - categorical_accuracy: 0.7059
Epoch 147/2000
3/3 [==============================] - 2s 561ms/step - loss: 0.4074 - categorical_accuracy: 0.7765
Epoch 148/2000
3/3 [==============================] - 2s 569ms/step - loss: 0.3991 - categorical_accuracy: 0.7882
Epoch 149/2000
3/3 [==============================] - 2s 589ms/step - loss: 0.3884 - categorical_accuracy: 0.8000
Epoch 150/2000
3/3 [==============================] - 2s 605ms/step - loss: 0.3890 - categorical_accuracy: 0.7882
Epoch 151/2000
3/3 [==============================] - 2s 536ms/step - loss: 0.7315 - categorical_accuracy: 0.6824
Epoch 152/2000
3/3 [==============================] - 2s 536ms/step - loss: 0.4479 - categorical_accuracy: 0.7412
Epoch 153/2000
3/3 [==============================] - 2s 548ms/step - loss: 0.4726 - categorical_accura

KeyboardInterrupt: 

In [17]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 30, 64)            442112    
                                                                 
 lstm_1 (LSTM)               (None, 30, 128)           98816     
                                                                 
 lstm_2 (LSTM)               (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 3)                 99        
                                                                 
Total params: 596,675
Trainable params: 596,675
Non-trai

# Make predictions

In [18]:
res = model.predict(X_test)

In [19]:
actions[np.argmax(y_test[4])]

'hello'

In [20]:
actions[np.argmax(res[4])]

'hello'

In [21]:
actions[np.argmax(y_test[3])]

'thanks'

In [22]:
actions[np.argmax(res[3])]

'iloveyou'

# Save weights

In [1]:
model.save('action.h5')

NameError: name 'model' is not defined

In [2]:
from tensorflow import keras

In [3]:
model1 = keras.models.load_model("action.h5")

# Evaluation using Confusion Matrix and Accuracy

In [21]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [22]:
yhat = model1.predict(X_test)

In [23]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [24]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[3, 1],
        [0, 1]],

       [[1, 0],
        [1, 3]]], dtype=int64)

In [25]:
accuracy_score(ytrue, yhat)

0.8

In [26]:
ytrue

[2, 2, 0, 2, 2]

In [27]:
yhat

[0, 2, 0, 2, 2]

# Test in real time

In [45]:
sequence=[]
sentence=[]
threshold=0.4

cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5,min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        ret, frame = cap.read()

        #make detections
        image,results = mediapipe_detection(frame,holistic)
        
        #draw landmarks
#         draw_landmarks(image,results)
        draw_styled_landmarks(image,results)
        
        #prediction login
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]
        if len(sequence)==30:
            res = model1.predict(np.expand_dims(sequence,axis=0))[0]
            
        #viz logic
        if res[np.argmax(res)]>threshold:
            if len(sentence)>0:
                if actions[np.argmax(res)] != sentence[-1]:
                    sentence.append(actions[np.argmax(res)])
            else:
                sentence.append(actions[np.argmax(res)])
                    
        if len(sentence)>5:
            sentence = sentence[-5:]
        
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)
            
        #show to screen
        cv2.imshow('OpenCV Feed',image)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
cap.release()
cv2.destroyAllWindows()

In [16]:
from scipy import stats

colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [ ]:
# 1. New detection variables
sequence = []
sentence = []
predictions = []
threshold = 0.5

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
#         print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model1.predict(np.expand_dims(sequence, axis=0))[0]
#             print(actions[np.argmax(res)])
            predictions.append(np.argmax(res))
            
            
        #3. Viz logic
            if np.unique(predictions[-10:])[0]==np.argmax(res): 
                if res[np.argmax(res)] > threshold: 
                    
                    if len(sentence) > 0: 
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Viz probabilities
            image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()